In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils import data

from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

device = torch.device("mps" if torch.has_mps else "cpu")
print(device)
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from aif360.algorithms.postprocessing import (CalibratedEqOddsPostprocessing,
                                              EqOddsPostprocessing,
                                              RejectOptionClassification)
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
random_state = 1

mps


pip install 'aif360[AdversarialDebiasing]'


In [2]:
# read in data and split
X = torch.load('inputs/logistic_regression_inputs.pt').cpu()
y = torch.load('inputs/logistic_regression_labels.pt').cpu()
df = pd.read_csv('inputs/logistic_regression_df.csv')
gender = df['reference_gender']
ethnicity = df['reference_ethnicity']
#extract asian and white data points
df = df[(ethnicity=='asian') | (ethnicity=='white')].reset_index()
X = X[(ethnicity=='asian') | (ethnicity=='white')]
y = y[(ethnicity=='asian') | (ethnicity=='white')]
gender = gender[(ethnicity=='asian') | (ethnicity=='white')].to_numpy()
ethnicity = ethnicity[(ethnicity=='asian') | (ethnicity=='white')].to_numpy()

df

,index,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
0,0,n004721,n004721,asian,asian,male,male,1.0
1,1,n004721,n004721,asian,asian,male,male,1.0
2,2,n004721,n004721,asian,asian,male,male,1.0
3,3,n004721,n004721,asian,asian,male,male,1.0
4,4,n004721,n004721,asian,asian,male,male,1.0
...,...,...,...,...,...,...,...,...
19133,38394,n004791,n008431,asian,asian,male,male,0.0
19134,38395,n003412,n005685,white,white,male,male,0.0
19135,38397,n008452,n008150,asian,asian,male,male,0.0
19136,38398,n004936,n002509,asian,asian,female,female,0.0


In [3]:
train_split, test_split = train_test_split(np.arange(len(X)),test_size=0.2, random_state=random_state)
train_split, val_split = train_test_split(train_split,test_size=0.25, random_state=random_state)
X_train = X[train_split]
X_val = X[val_split]
X_test = X[test_split]
y_train = y[train_split]
y_val = y[val_split]
y_test = y[test_split]

ethnicity_train = ethnicity[train_split]
ethnicity_train[ethnicity_train=='white'] = 0
ethnicity_train[ethnicity_train=='asian'] = 1
ethnicity_train = ethnicity_train.astype(int)

ethnicity_val = ethnicity[val_split]
ethnicity_val[ethnicity_val=='white'] = 0
ethnicity_val[ethnicity_val=='asian'] = 1
ethnicity_val = ethnicity_val.astype(int)

ethnicity_test = ethnicity[test_split]
ethnicity_test[ethnicity_test=='white'] = 0
ethnicity_test[ethnicity_test=='asian'] = 1
ethnicity_test = ethnicity_test.astype(int)

In [4]:
train_df = df.iloc[train_split]
val_df = df.iloc[val_split]
test_df = df.iloc[test_split]
test_df

,index,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
4535,9335,n004477,n004477,white,white,male,male,1.0
4471,9271,n002343,n002343,white,white,male,male,1.0
506,506,n009143,n009143,asian,asian,male,male,1.0
35,35,n003355,n003355,asian,asian,male,male,1.0
3661,8461,n000545,n000545,white,white,male,male,1.0
...,...,...,...,...,...,...,...,...
6774,11574,n000160,n000160,asian,asian,female,female,1.0
14955,30026,n008069,n009147,white,white,female,female,0.0
10355,20738,n005174,n008109,white,white,female,female,0.0
8786,18386,n005385,n005385,white,white,female,female,1.0


In [5]:
## train data
class TrainData(data.Dataset):
    
    def __init__(self, X_data, y_data,gender):
        self.X_data = X_data
        self.y_data = y_data
        self.gender = gender    
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index], self.gender[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train),
                       ethnicity_train)
test_data = TrainData(torch.FloatTensor(X_test),torch.FloatTensor(y_test),ethnicity_test)
val_data = TrainData(torch.FloatTensor(X_val), 
                       torch.FloatTensor(y_val),
                       ethnicity_val)
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 4096
        self.layer_1 = nn.Linear(4096, 1024) 
        self.layer_2 = nn.Linear(1024, 512)
        self.layer_out = nn.Linear(512, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(1024)
        self.batchnorm2 = nn.BatchNorm1d(512)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

def confusion_mat(y_pred, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tn + tp)/(tn+tp+fn+fp)
    return tn, fp, fn, tp, acc
    
def AOE(tn_1,fp_1,fn_1,tp_1,tn_0,fp_0,fn_0,tp_0):
    tpr_1 = tp_1/(tp_1+fn_1)
    tpr_0 = tp_0/(tp_0+fn_0)

    fpr_1 = fp_1/(fp_1+tn_1)
    fpr_0 = fp_0/(fp_0+tn_0)


    return (np.abs(fpr_1-fpr_0) + np.abs(tpr_1 - tpr_0))/2


In [6]:
model = torch.load('weights/asian_white_logistic_regression_face_matching.pt')

In [7]:
model.to(device)
criterion = nn.BCEWithLogitsLoss()
test_loader = data.DataLoader(dataset=test_data, batch_size=12)
model.eval()
test_labels = []
test_predictions = []
test_ethnicities = []
with torch.no_grad():
    for X_test, y_test, ethnicity_test in test_loader:
        X_test, y_test = X_test.to(device), y_test.to(device)
        test_out = model(X_test)
        test_pred = torch.round(torch.sigmoid(test_out))
        test_labels.append(y_test)
        test_predictions.append(test_pred)
        test_ethnicities.append(ethnicity_test)
test_labels = torch.cat(test_labels)
test_predictions = torch.cat(test_predictions).reshape(-1)
test_ethnicities = torch.cat(test_ethnicities)

In [8]:
test_bias_df = pd.DataFrame(columns=['label','prediction','protected'])
test_bias_df['label']=test_labels.cpu().numpy()
test_bias_df['prediction']=test_predictions.cpu().numpy()
test_bias_df['protected']=test_ethnicities.cpu().numpy()
test_bias_df

,label,prediction,protected
0,1.0,1.0,0
1,1.0,1.0,0
2,1.0,1.0,1
3,1.0,1.0,1
4,1.0,1.0,0
...,...,...,...
3823,1.0,1.0,1
3824,0.0,0.0,0
3825,0.0,0.0,0
3826,1.0,1.0,0


In [9]:
unpriveleged_df = test_bias_df[test_bias_df['protected']==1]
unpriveleged_df

,label,prediction,protected
2,1.0,1.0,1
3,1.0,1.0,1
8,0.0,0.0,1
10,0.0,0.0,1
11,0.0,0.0,1
...,...,...,...
3814,1.0,1.0,1
3818,1.0,1.0,1
3820,1.0,1.0,1
3822,1.0,1.0,1


In [10]:
priveleged_df = test_bias_df[test_bias_df['protected']==0]
priveleged_df

,label,prediction,protected
0,1.0,1.0,0
1,1.0,1.0,0
4,1.0,1.0,0
5,1.0,1.0,0
6,1.0,1.0,0
...,...,...,...
3821,1.0,1.0,0
3824,0.0,0.0,0
3825,0.0,0.0,0
3826,1.0,1.0,0


In [182]:
unpriveleged_tn, unpriveleged_fp, unpriveleged_fn, unpriveleged_tp, unpriveleged_acc = confusion_mat(unpriveleged_df['prediction'], unpriveleged_df['label'])
priveleged_tn, priveleged_fp, priveleged_fn, priveleged_tp, priveleged_acc = confusion_mat(priveleged_df['prediction'], priveleged_df['label'])
unpriveleged_fnr = unpriveleged_fn/(unpriveleged_fn+unpriveleged_tp)
unpriveleged_fpr = unpriveleged_fp/(unpriveleged_fp+unpriveleged_tn)
priveleged_fnr = priveleged_fn/(priveleged_fn+priveleged_tp)
priveleged_fpr = priveleged_fp/(priveleged_fp+priveleged_tn)

print('Overall Accuracy:',round(torch.sum(test_predictions == test_labels).item()/test_labels.size()[0],3))
print('Missclassificaition Rate Ratio:',round((1-unpriveleged_acc)/(1- priveleged_acc),3))
print('FNR Ratio:',round(unpriveleged_fnr/priveleged_fnr,3),'FPR Ratio:',round(unpriveleged_fpr/priveleged_fpr,3))
print('AOE:',round(AOE(unpriveleged_tn,unpriveleged_fp,unpriveleged_fn,unpriveleged_tp,priveleged_tn,priveleged_fp,priveleged_fn,priveleged_tp),3))

Overall Accuracy: 0.955
Missclassificaition Rate Ratio: 1.901
FNR Ratio: 1.956 FPR Ratio: 1.847
AOE: 0.028


In [50]:
### AIF implementations
def val(model,dataloader):
    y_true, y_pred, y_prot = [],[],[]
    model.eval()
    with torch.no_grad():
        for X_val, y_val,prot in dataloader:
            X_val, y_val = X_val.to(device), y_val.to(device)
            y_pred.append(torch.sigmoid(model(X_val)[:, 0]))
            y_true.append(y_val)
            y_prot.append(prot)
    y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)
    return y_true, y_pred, y_prot

def to_dataframe(y_true, y_pred, y_prot):
        y_true, y_pred, y_prot = y_true.float().cpu().numpy(), y_pred.float().cpu().numpy(), y_prot.float().cpu().numpy()
        df = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred, 'y_prot': y_prot})
        dataset = StandardDataset(df, 'y_true', [1.], ['y_prot'], [[1.]])
        dataset.scores = y_pred.reshape(-1, 1)
        return dataset

In [145]:
y_true, y_pred, y_prot = val(model,test_loader)
aif_data = to_dataframe(y_true, y_pred, y_prot)


In [216]:
cost_constraint = 'weighted'
privileged_groups=[{'y_prot': 0.}],
unprivileged_groups=[{'y_prot': 1.}]
cpp = CalibratedEqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                             unprivileged_groups=[{'y_prot': 1.}],
                                             cost_constraint=cost_constraint)
cpp=cpp.fit(aif_data,aif_data)
cpp_pred = cpp.predict(aif_data).labels.reshape(-1)
cpp_label = aif_data.labels.reshape(-1)
cpp_prot = aif_data.protected_attributes.reshape(-1)

unpriv_ids = (cpp_prot==1)
unpriv_pred = cpp_pred[unpriv_ids]
unpriv_label = cpp_label[unpriv_ids]

priv_ids = (cpp_prot==0)
priv_pred = cpp_pred[priv_ids]
priv_label = cpp_label[priv_ids]

unpriv_tn, unpriv_fp, unpriv_fn, unpriv_tp, unpriv_acc = confusion_mat(unpriv_pred, unpriv_label)
unpriv_fpr = unpriv_fp/(unpriv_fp+unpriv_tn)
unpriv_fnr = unpriv_fn/(unpriv_fn+unpriv_tp)

priv_tn, priv_fp, priv_fn, priv_tp, priv_acc = confusion_mat(priv_pred, priv_label)
priv_fpr = priv_fp/(priv_fp+priv_tn)
priv_fnr = priv_fn/(priv_fn+priv_tp)

print('Missclassificaition Rate Ratio:',round((1-unpriv_acc)/(1- priv_acc),3))
print('FPR Ratio:',round(unpriv_fpr/priv_fpr,3))
print('FNR Ratio:',round(unpriv_fnr/priv_fnr,3))
print('Overall Accuracy:',round(np.sum(cpp_pred == cpp_label)/len(cpp_label),3))
print('AOE:',round(AOE(unpriv_tn, unpriv_fp, unpriv_fn, unpriv_tp,priv_tn, priv_fp, priv_fn, priv_tp),3))

Missclassificaition Rate Ratio: 0.984
FPR Ratio: 0.706
FNR Ratio: 1.956
Overall Accuracy: 0.94
AOE: 0.027


In [218]:
eo = EqOddsPostprocessing(privileged_groups=[{'y_prot': 0.}],
                                  unprivileged_groups=[{'y_prot': 1.}])

eo=eo.fit(aif_data,aif_data)

eo_pred = eo.predict(aif_data).labels.reshape(-1)
eo_label = aif_data.labels.reshape(-1)
eo_prot = aif_data.protected_attributes.reshape(-1)

eo_unpriv_ids = (eo_prot==1)
eo_unpriv_pred = eo_pred[eo_unpriv_ids]
eo_unpriv_label = eo_label[eo_unpriv_ids]

eo_priv_ids = (eo_prot==0)
eo_priv_pred = eo_pred[eo_priv_ids]
eo_priv_label = eo_label[eo_priv_ids]

eo_unpriv_tn, eo_unpriv_fp, eo_unpriv_fn, eo_unpriv_tp, eo_unpriv_acc = confusion_mat(eo_unpriv_pred, eo_unpriv_label)
eo_unpriv_fpr = eo_unpriv_fp/(eo_unpriv_fp+eo_unpriv_tn)
eo_unpriv_fnr = eo_unpriv_fn/(eo_unpriv_fn+eo_unpriv_tp)

eo_priv_tn, eo_priv_fp, eo_priv_fn, eo_priv_tp, eo_priv_acc = confusion_mat(eo_priv_pred, eo_priv_label)
eo_priv_fpr = eo_priv_fp/(eo_priv_fp+eo_priv_tn)
eo_priv_fnr = eo_priv_fn/(eo_priv_fn+eo_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-eo_unpriv_acc)/(1- eo_priv_acc),3))
print('FPR Ratio:',eo_unpriv_fpr/eo_priv_fpr)
print('FNR Ratio:',eo_unpriv_fnr/eo_priv_fnr)
print(np.sum(eo_pred == eo_label)/len(eo_pred))
print('AOE:',round(AOE(eo_unpriv_tn, eo_unpriv_fp, eo_unpriv_fn, eo_unpriv_tp,eo_priv_tn, eo_priv_fp, eo_priv_fn, eo_priv_tp),3))


Missclassificaition Rate Ratio: nan
FPR Ratio: nan
FNR Ratio: nan
1.0
AOE: 0.0


In [225]:
ROC = RejectOptionClassification(unprivileged_groups=[{'y_prot': 1.}],
                                         privileged_groups=[{'y_prot': 0.}],
                                         low_class_thresh=0.01, high_class_thresh=0.99,
                                         num_class_thresh=100, num_ROC_margin=100,
                                         metric_name="Average odds difference",
                                         metric_ub=0.01, metric_lb=-0.01)

ROC = ROC.fit(aif_data, aif_data)

ROC_pred = ROC.predict(aif_data).labels.reshape(-1)
ROC_label = aif_data.labels.reshape(-1)
ROC_prot = aif_data.protected_attributes.reshape(-1)

ROC_unpriv_ids = (ROC_prot==1)
ROC_unpriv_pred = ROC_pred[ROC_unpriv_ids]
ROC_unpriv_label = ROC_label[ROC_unpriv_ids]

ROC_priv_ids = (ROC_prot==0)
ROC_priv_pred = ROC_pred[ROC_priv_ids]
ROC_priv_label = ROC_label[ROC_priv_ids]

ROC_unpriv_tn, ROC_unpriv_fp, ROC_unpriv_fn, ROC_unpriv_tp, ROC_unpriv_acc = confusion_mat(ROC_unpriv_pred, ROC_unpriv_label)
ROC_unpriv_fpr = ROC_unpriv_fp/(ROC_unpriv_fp+ROC_unpriv_tn)
ROC_unpriv_fnr = ROC_unpriv_fn/(ROC_unpriv_fn+ROC_unpriv_tp)

ROC_priv_tn, ROC_priv_fp, ROC_priv_fn, ROC_priv_tp, ROC_priv_acc = confusion_mat(ROC_priv_pred, ROC_priv_label)
ROC_priv_fpr = ROC_priv_fp/(ROC_priv_fp+ROC_priv_tn)
ROC_priv_fnr = ROC_priv_fn/(ROC_priv_fn+ROC_priv_tp)

print('Missclassificaition Rate Ratio:',round((1-ROC_unpriv_acc)/(1- ROC_priv_acc),3))
print('FPR Ratio:',round(ROC_unpriv_fpr/ROC_priv_fpr,3))
print('FNR Ratio:',round(ROC_unpriv_fnr/ROC_priv_fnr,3))
print(np.sum(ROC_pred == ROC_label)/len(ROC_pred))
print('AOE:',round(AOE(ROC_unpriv_tn, ROC_unpriv_fp, ROC_unpriv_fn, ROC_unpriv_tp,ROC_priv_tn, ROC_priv_fp, ROC_priv_fn, ROC_priv_tp),3))


Missclassificaition Rate Ratio: 1.95
FPR Ratio: 3.41
FNR Ratio: 1.533
0.9584639498432602
AOE: 0.027


In [223]:
y_pred

tensor([0.9906, 0.9974, 0.9640,  ..., 0.1211, 0.9921, 0.3239], device='mps:0')